In [19]:
import numpy as np
from scipy.optimize import fsolve

In [20]:
# Datos del problema
#                 A         B        C       D        E
Antoine = {1: [58.302, -5385.6, -5.4954, 5.3634e-6, 2],
           2: [154.62, -8793.1, -21.684, 2.3916e-2, 1]}

Pressure = 1e5       # Presion total del sistema, Pa
alpha = 0.333        # Parametro de NRTL adimensional
b12_R = 768.57         # Parametro energetico de NTRL, K
b21_R = 465.43         # Parametro energetico de NTRL, K

bs_R = [b12_R, b21_R]

# Existe algun azeotropo en el sistema?

Como los coeficientes de actividades de los componentes puede ser modelado empleando las correlaciones de NRTL, entonces es posible que haya azeotropo a la presion de 1 bar. Pero existira? a cual temperatura?

Para saber si hay un azeótropo se calcula los valores de la volatilidad relativa del etanol a sus valores extremos de composión.

De esta manera se debe evaluar las siguientes relaciones:

\begin{equation*}
\lim_{x_1 \to 1} \alpha_{12} = \frac{P_1^\text{sat}(T_1^\text{sat})}{\gamma_2^\infty P_2^\text{sat}(T_1^\text{sat})} \qquad \lim_{x_1 \to 0} \alpha_{12} = \frac{\gamma_1^\infty P_1^\text{sat}(T_2^\text{sat})}{P_2^\text{sat}(T_2^\text{sat})}
\end{equation*}

Si hay un cambio en el valor de la volatilidad relativa con respecto a 1, entonces habra al menos azeótropo en la solución.

Para realizar el calculo se necesita:

- Expresion para calcular la presion de saturacion
- Expresion para determinar los valores de los coeficientes de actividad a dilucion infinita

La presión de saturación puede ser estimada mediante:

\begin{equation}
P^\text{sat} = \exp\left(A + \frac{B}{T} + C \, \ln(T) + D \, T^E\right)
\end{equation}

Mientras que para evaluar los coeficientes de actividad a dilucion infinita se tiene:

$$
\begin{aligned}
&\tau_{12} = \frac{b_{12}}{R\:T} \\
&\tau_{21} = \frac{b_{21}}{R\:T} \\
&\ln\left(\gamma_{1}^\infty\right) = \tau_{21} + \tau_{12} \: \exp(-\alpha  \tau_{12}) \\
&\ln\left(\gamma_{2}^\infty\right) = \tau_{12} + \tau_{21} \: \exp(-\alpha  \tau_{21})
\end{aligned}
$$



In [21]:
# Vamos a crear algunas funciones auxiliares
def psat(temp, coef_antoine):
    """
    Calcula presion de saturación de un componente puro para un presión dada.
    input:
        temp: temperatura del sistema, K
        coef_antoine: Parametros de Ecuacion de Antoine
    output:
        presion de saturacion en Pascal
    """
    a, b, c, d, e = coef_antoine
    return np.exp(a + b/temp + c * np.log(temp) + d * temp**e)

def tsat(temp, *args):
    """
    Calcula la temperatura de saturacion de un componente

    Input:
        presion (float): Presion total del sistema, Pa
        coef_antoine (float): Parametros de la ecuacion de Antoine del componente
    Output:
       temperatura de saturacion del componente, K
    """
    presion, coef_antoine = args
    return presion - psat(temp, coef_antoine)


In [22]:

def coef_dilucion_infinita(press, alpha, bs_r, antoine):
    """
    Calcula la volatilidad relativa 12 cuando la solucion
    es concentrada y a dilucion infinita
    input:
        press: presion total de sistema, Pa
        alpha: parametro de NRTL para sistema termodinamica, adim
        bs_r: parametro energetico para comp12 y comp21, K
        antoine: parametros de ecuacion de Antoine
    output:
        alpha_12_infty: volatilidad relativa 12, adim
        msg: string que indica si hay azeotropo en el sistema
        tsats: temperaturas de saturacion de los componentes puros, K
    """
        
    tsats = [fsolve(tsat, 300, args=(press, antoine[i]))[0] for i in antoine.keys()]
    alpha_12_infty = []
    exponencial = {0:-1, 1:1}
    
    for i in range(len(tsats)):
        # cuando i = 0 => solucion a dilucion infinita en componente 2
        #        i = 1 => solucion a dilucion infinita en componente 1
        taus = bs_r / tsats[1-i]
        gamma_infty = np.exp(taus[i] + taus[1-i] * np.exp(-alpha * taus[1-i]))
        psat1, psat2 = psat(tsats[i], antoine[1]), psat(tsats[i], antoine[2])
        alpha_12_infty.append(gamma_infty**exponencial[i] * psat1/psat2)
    
    alpha_12_infty = np.array(alpha_12_infty)
    if ((alpha_12_infty >= 1).sum() != alpha_12_infty.size).astype(np.int32):
        if alpha_12_infty[0] > 1:
            msg = 'El sistema presenta un azetropo de maxima ebullicion'
        else:
            msg = 'El sistema presenta un azetropo de minima ebullicion'
    else:
        msg = 'El sistema no presenta punto azeotropico'
    
    return (alpha_12_infty, msg, tsats)


In [23]:
alpha12, mensaje, Tsats = coef_dilucion_infinita(Pressure, alpha, bs_R, Antoine)
print(f"Con base a los valores de volatidad relativa 12 se concluye que: \n%s"%(mensaje))

Con base a los valores de volatidad relativa 12 se concluye que: 
El sistema presenta un azetropo de minima ebullicion


## Ubicacion del punto azeotropico

Ahora que se conoce que el sistema presenta un azeotropo de ebullicion minima, entonces se debe determinar donde se encuentra antes de realizar calculos de equilibrio de fases para:
1. Determinar si es factible fisicamente realizar la separacion de los compuestos
2. Evitar complicaciones innecesarias en los calculos numericos

Para calcular el lugar exacto donde ocurre el azeotropo. Lo que se debe resolver son las ecuaciones de ELV con la restriccion que la composicion de cada uno de los compuestos sera igual a la de su fase complementaria. 
 
 $$
 P = \gamma_i \: P_i^\text{sat}
$$
 
 Los coeficientes de actividad deben calcularse con el modelo de NRTL
 
 $$
 \begin{aligned}
 &\ln \gamma _1  = x_2^2\,\left[ \tau_{21}\left(\frac{G_{21}}{x_1+x_2G_{21}}\right)^2 + \frac{\tau_{12}G_{12}}{\left(x_2 + x_1 G_{12}\right)^2} \right] \\
 &\ln \gamma _2  = x_1^2\,\left[ \tau_{12}\left(\frac{G_{12}}{x_2+x_1G_{12}}\right)^2 + \frac{\tau_{21}G_{21}}{\left(x_1 + x_2 G_{21}\right)^2} \right] \\
 &\tau_{12} = \frac{b\,_{12}}{T} \qquad \tau_{21} = \frac{b\,_{21}}{T} \\
 &G_{12} =\exp\left(-\alpha\,\tau_{12}\right) \qquad G_{21} =\exp\left(-\alpha \,\tau_{21}\right)
 \end{aligned}
 $$

 El sistema de ecuaciones a resolver seria:

 $$
 \begin{align*}
 P - \gamma_1 (x_1, T) \: P_1^\text{sat}(T) = 0\\
 P - \gamma_2 (x_1, T) \: P_2^\text{sat}(T) = 0  
 \end{align*}
 $$
 
 Siendo las incognitas $x_1$ y $T$ 

In [24]:

def sistema_azeotropico(incognitas, bs_r, pressure, alpha, antoine):
    """
    Calcula temperatura y composición en el punto azeotrópico.
    input:
        incognitas: los valores semillas de la composicion del compuesto 1
                    y la temperatura en el punto azeotropico
        bs_r: parametro energetico para comp12 y comp21, K
        pressure: presion total de sistema, Pa
        alpha: parametro de NRTL para sistema termodinamica, adim
        antoine: parametros de ecuacion de Antoine
    output:
        f: composicion y temperatura en el punto azeotropico
    """
    x1, temp = incognitas

    taus = bs_r / temp         # tau12, tau21
    
    gs = np.exp(-alpha * taus) # G12, G21
    
    x = [x1, 1 - x1]           # Composicion 1, Composicion 2
    
    ln_gamma = []
    psats = []
    for i in range(len(x)):
        ln_gamma.append(x[1-i]**2 * (taus[1-i] * (gs[1-i]/(x[i] + x[1-i] * gs[1-i]))**2 + 
                                     taus[i] * gs[i] / (x[1-i] + x[i] * gs[i])**2))
        psats.append(psat(temp, antoine[i+1]))
    
    f = pressure - np.exp(ln_gamma) * np.array(psats)
    
    return f

Como semillas consideramos que la fraccion del azeótropo es 0.5 y la temperatura del sistema debe ser menor que la temperatura de saturacion del componente mas volatil

In [25]:
Tsat1, Tsat2 = Tsats
print("La temperatura de saturación del acetonitrilo (1) es {:.3f} K, "
     "mientras que la del n-heptano (2) es {:.3f} K \ncuando la presion "
     "del sistema es igual a {:.0f} bar".format(Tsat1, Tsat2, Pressure/1e5))

La temperatura de saturación del acetonitrilo (1) es 354.208 K, mientras que la del n-heptano (2) es 371.276 K 
cuando la presion del sistema es igual a 1 bar


In [26]:
valores_semillas = (0.5, Tsats[0] * .9)
x1_azeotropo, Temp_azeotropo = fsolve(sistema_azeotropico, valores_semillas, 
                                      args=(bs_R, Pressure, alpha, Antoine))

print('El punto azeotrópico ocurre a una fraccion molar de {:.3f} y una '
      'temperatura de {:.2f} K'.format(x1_azeotropo, Temp_azeotropo))

El punto azeotrópico ocurre a una fraccion molar de 0.644 y una temperatura de 341.85 K


## Determinación de temperatura de operación del flash

Como se conoce la fracción del acetonitrilo (1) en la fase vapor, el problema se reduce a resolver el mismo sistema de ecuaciones anteriores con la diferencia que se debe anadir la composicion en la fase vapor como argumento:

\begin{align*}
&y_1 \, P - \gamma_1 (x_1, T) \: P_1^\text{sat}(T) \, x_1 = 0\\
&(1 - y_1) \, P - \gamma_2 (x_1, T) \: P_2^\text{sat}(T) \, (1 - x_1) = 0  
\end{align*}

Siendo las incognitas $x_1$ y $T$

Con respecto a los valores semillas, tanto la composicion del componente 1 en la fase liquida debe ser menor que la del punto azeotropico, mientras que la temperatura del sistema deben ser mayor.

De esta manera, supondremos que x1 es el 10% de la composicion del punto azeotropico y la temperatura de operacion sera un 20% mayor

In [27]:
def sistema_flash(incognitas, composicion_vapor, bs_r, pressure, alpha, antoine):
    """
    Calcula temperatura y composición en el punto azeotrópico.
    input:
        incognitas: los valores semillas de la composicion del compuesto 1
                    y la temperatura en equilibrio
        composicion_vapor: composicion del componente 1 en la fase vapor
        bs_r: parametro energetico para comp12 y comp21, K
        pressure: presion total de sistema, Pa
        alpha: parametro de NRTL para sistema termodinamica, adim
        antoine: parametros de ecuacion de Antoine
    output:
        f: composicion y temperatura en equilibrio de fases
    """
    x1, temp = incognitas

    taus = bs_r / temp         # tau12, tau21
    
    gs = np.exp(-alpha * taus) # G12, G21
    
    x = np.array([x1, 1 - x1])           # Composicion 1, Composicion 2
    y = np.array([composicion_vapor, 1 - composicion_vapor])
    
    ln_gamma = []
    psats = []
    for i in range(len(x)):
        ln_gamma.append(x[1-i]**2 * (taus[1-i] * (gs[1-i]/(x[i] + x[1-i] * gs[1-i]))**2 + 
                                     taus[i] * gs[i] / (x[1-i] + x[i] * gs[i])**2))
        psats.append(psat(temp, antoine[i+1]))
    
    f = y * pressure - x * np.exp(ln_gamma) * np.array(psats)
    
    return f

In [28]:
y1 = 0.85 * x1_azeotropo
valores_semillas = np.array([x1_azeotropo, Temp_azeotropo]) * [0.1, 1.2]

x1_flash, Temp_flash = fsolve(sistema_flash, valores_semillas, args=(y1, bs_R, Pressure, alpha, Antoine))

print(('Si el flash opera a {:.2f} K\n'
       'se obtendra una composicion de 1 en la fase liquida de {:.2f}%').format(Temp_flash, x1_flash*100))


Si el flash opera a 348.59 K
se obtendra una composicion de 1 en la fase liquida de 9.26%
